In [10]:
import pandas as pd
import numpy as np
import json
import codecs

In [11]:
from tqdm import tqdm

In [12]:
import random

In [13]:
from src.refine_text import normalize_golden_text

In [14]:
file_path = '/Users/bryan/Downloads/vinmec_report_80k_unique_lines.txt'

In [15]:
with open(file_path, 'rt') as f:
    lines = [line.rstrip() for line in f]

In [16]:
gold_data_refined = []
gold_data_raw = []

for line in tqdm(lines):
    try:
        content = normalize_golden_text(line)
    except:
        print(line)
        break
    if len(content.split()) > 5:
        gold_data_refined.append(content)
        gold_data_raw.append(line)

100%|██████████| 79369/79369 [00:25<00:00, 3152.29it/s]


In [17]:
export_data = pd.DataFrame(zip(gold_data_refined, gold_data_raw), columns=['refined', 'raw'])

In [18]:
export_data.to_csv('/Users/bryan/Desktop/export_data_common_voices.csv', encoding='utf-8-sig')

In [ ]:
from string import Template

In [ ]:
example = Template('$verb $sentence_loc $prep $what')

In [ ]:
example.substitute(verb='chỉnh', sentence_loc='câu 7', prep='thành', what='tim to')

In [ ]:
templates = {
  "modify": {
    "sentences": [
      "$verb $sentence_loc $prep $what",
      "$sentence_loc $verb $prep $what"
    ],
    "verb": [
      "chỉnh",
      "sửa",
      "đổi",
      "thay"
    ],
    "sentence_loc": [
      "trước câu $number",
      "sau câu $number",
      "trước câu số $number",
      "sau câu số $number",
      "câu số $number",
      "hàng số $number",
      "dòng số $number",
      ""
    ],
    "prep": [
      "thành",
      "bằng",
      "về",
    ]
  },
  "delete": {
    "sentences": [
      "$verb $sentence_loc"
    ],
    "verb": [
      "loại bỏ",
      "xóa",
      "bỏ",
      "hủy"
    ],
    "sentence_loc": [
      "câu $number"
    ]
  },
  "add": {
    "sentences": [
      "$verb $prep $sentence_loc $what",
      "$verb $what $prep $sentence_loc",
      "$what $verb $prep $sentence_loc"
    ],
    "verb": [
      "thêm",
      "bổ sung",
      "nhập",
      "chèn"
    ],
    "prep": [
      "vào trước",
      "vào sau",
      "vào dưới",
      "vào trên",
    ],
    "sentence_loc": [
      "câu $number",
      "dòng $number",
      "hàng $number"
    ]
  }
}

In [ ]:
with open('/Users/bryan/Desktop/template.json', 'w') as outfile:
    json.dump(templates, outfile)

In [ ]:
prob_add, prob_del, prob_modify = random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)

In [ ]:
ind_case = np.argmax([prob_add, prob_del, prob_modify])
if ind_case == 0:
    case = 'modify'
elif ind_case == 1:
    case = 'delete'
else:
    case = 'add'
case

In [ ]:
number = random.randint(1, 10)
example = random.sample(templates[case]['sentences'], 1)[0]
verb = random.sample(templates[case]['verb'], 1)[0]
prep = random.sample(templates[case]['prep'], 1)[0]
what = random.sample(gold_data_refined, 1)[0]
sentence_loc = random.sample(templates[case]['sentence_loc'], 1)[0]

In [ ]:
example, verb, prep, what, sentence_loc

In [ ]:
example = Template(example)
sentence_loc = Template(sentence_loc)
sentence_loc = sentence_loc.substitute(number=number)
text = example.substitute({'verb': verb, 'sentence_loc': sentence_loc, 'prep': prep, 'what': what})

In [ ]:
text

In [ ]:
def generate_text_voice(templates, finding:str) -> str:
    prob_add, prob_del, prob_modify, prob_nothing = random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)
    ind_case = np.argmax([prob_add, prob_del, prob_modify, prob_nothing])
    if ind_case == 0:
        case = 'modify'
    elif ind_case == 1:
        case = 'delete'
    elif ind_case == 2:
        case = 'add'
    else:
        case = 'nothing'
    if templates.get(case, None) is not None:
        if templates[case].get('sentences') is not None:
            example = random.sample(templates[case]['sentences'], 1)[0]
        else:
            print(f'case {case} has error on sentences')
            
        if templates[case].get('verb') is not None:
            verb = random.sample(templates[case]['verb'], 1)[0]
        else:
            print(f'case {case} has error on verb')
            
        if templates[case].get('prep') is not None:
            prep = random.sample(templates[case]['prep'], 1)[0]
        else:
            print(f'case {case} has error on prep')
        
        if templates[case].get('sentence_loc') is not None:
            sentence_loc = random.sample(templates[case]['sentence_loc'], 1)[0]
        else:
            print(f'case {case} has error on sentence_loc')
        
        number = random.randint(1, 10)
        example = Template(example)
        sentence_loc = Template(sentence_loc)
        sentence_loc = sentence_loc.substitute(number=number)
        text = example.substitute(verb=verb, sentence_loc=sentence_loc, prep=prep, what=finding)
        return text, case, number
    else:
        return finding, None, None

In [ ]:
generate_text_voice(templates, random.sample(gold_data_refined, 1)[0])

In [ ]:
templates = json.loads(codecs.open('/Users/bryan/Desktop/template.json', 'r', 'UTF-8').read())

In [ ]:
templates

In [ ]:
generate_text_voice(templates, random.sample(gold_data_refined, 1)[0])

In [ ]:
foo = pd.read_excel('/Users/bryan/Downloads/Common Voice List_Vietnamese_100.xlsx')

In [ ]:
def run(template_path, data_path, n_samples):
    templates = json.loads(codecs.open(template_path, 'r', 'UTF-8').read())
    database = pd.read_excel(data_path)
    database = database['refined'].tolist()
    records = []
    for finding in tqdm(database):
        tmp = []
        for j in range(n_samples):
            rec_ = generate_text_voice(templates, finding)
            tmp.append(rec_)
        tmp = list(set(tmp))
        records = records + tmp
    return records

In [ ]:
bar = run('/Users/bryan/Desktop/template.json', '/Users/bryan/Downloads/Common Voice List_Vietnamese_100.xlsx', 2)

In [ ]:
templates['delete'].keys()

In [ ]:
example.substitute??